# MMPBSA method for protein-ligand binding free energy calculations

Xiping Gong (xipinggong@uga.edu) from the [Jack Huang's Lab](https://site.caes.uga.edu/huanglab/)

Date: 4/27/2025

# Introduction

Accurately estimating the binding free energy between a protein and a ligand is a crucial step in computational drug design and environmental toxicology studies. Molecular mechanics-based free energy calculations, such as Molecular Mechanics Poisson–Boltzmann Surface Area (MMPBSA) and Molecular Mechanics Generalized Born Surface Area (MMGBSA), provide a computationally efficient alternative to more expensive methods like Free Energy Perturbation (FEP) and Thermodynamic Integration (TI). While FEP and TI offer high accuracy by explicitly modeling the thermodynamic cycle of ligand binding, they require extensive sampling and computational resources. In contrast, MMPBSA and MMGBSA approximate binding free energies using an ensemble of molecular dynamics (MD) snapshots, significantly reducing computational cost while maintaining reasonable accuracy for ranking ligand affinities.

The [gmx_MMPBSA tool](https://pubs.acs.org/doi/full/10.1021/acs.jctc.1c00645) has enabled seamless integration of MMPBSA calculations into the GROMACS molecular dynamics platform, making it accessible for researchers studying protein-ligand interactions. This tool automates the extraction of trajectory frames and energy calculations, allowing users to efficiently estimate binding free energies within the GROMACS ecosystem.

In our previous study, we employed AlphaFold 3 and AutoDock Vina for molecular docking to predict the binding poses of per- and polyfluoroalkyl substances (PFAS) with target proteins (Link: https://xipinggong.com/files/tutorials/pfas_docking.html). While docking provides a rapid assessment of binding configurations, it does not account for entropic and solvation effects, which are critical for determining binding affinities. Therefore, we applied the MMPBSA method to further refine our binding free energy estimates and assess its suitability for studying protein-PFAS interactions.

In this tutorial, we present a step-by-step guide on how to use the gmx_MMPBSA tool to calculate protein-ligand binding free energies. We begin with an overview of the MMPBSA methodology, followed by a practical guide on running gmx_MMPBSA from a downloaded protein-ligand complex PDB file. This workflow serves as a resource for researchers interested in applying MMPBSA calculations to evaluate protein-ligand interactions efficiently within the GROMACS framework.



# Methodology

Here, we used the Single Trajectory (ST) protocol for MMPBSA calculations, extracting protein, ligand, and complex energies from the same MD trajectory to improve efficiency and consistency. However, the ST protocol has limitations, including restricted conformational sampling of both unbound and bound states. If your systems are disordered, then this ST protocol can be inaccurate. Normally, the implicit solvent models are often used to accelerate calculations while capturing key solvation effects, and gmx_MMPBSA allows decomposition into van der Waals, electrostatic, polar, and non-polar solvation contributions. However, one key component of implicit solvent models is to estimate the solvation free energy, where some PB and GB models are often used, but they are often relatively accurate. Despite these challenges, it remains a computationally efficient method for estimating protein-ligand binding free energy, making it valuable for large-scale screening and comparative studies. So, it is essential to do the assessment.

Given a configuration of protein-ligand system (e.g., $X$), we can introduce a reduced potential energy $W(X) = U_{p-p}(X) + G_{solv}(X)$, to estimate its free energy, where the $U_{p-p}(X)$ is the potential energy of solute system (like a protein-ligand system), and $G_{solv}(X)$ captures the solvent effects on the solute system. The $U_{p-p}(X)$ can be readily captured by the conventional mechanical force fields of protein-ligand systems, while the $G_{solv}(X)$ is often captured by the PB or GB implicit solvent models. In the ST protocol, two configurations are considered, including the unbound and bound configuration, where their only difference is whether protein and ligand systems are separated or not.

An illustrated description can be found in the following image:  
![Illustration of gmx_MMPBSA](https://pubs.acs.org/cms/10.1021/acs.jctc.1c00645/asset/images/medium/ct1c00645_0001.gif)  
*Source: [ACS JCTC](https://pubs.acs.org/doi/10.1021/acs.jctc.1c00645)*

where, the $\Delta G_{bind}$ can be represented by the $W(X_{\text{bound}}) - W(X_{\text{unbound}})$.
Since the Single Trajectory (ST) protocol is used, intra-molecular interactions within both the protein and ligand are ignored. Instead, only inter-molecular interactions between the protein and ligand are considered, which can be further categorized into non-polar interactions (e.g., van der Waals forces) and polar interactions (e.g., electrostatic interactions). We can have the following energy decomposition.

$\Delta G_{bind} = \Delta U_{\text{gas}} + \Delta G_{\text{solv}} $
$                = \Delta U_{\text{gas}}^{\text{vdW}} + \Delta U_{\text{gas}}^{\text{ele}} + $
$                  \Delta G_{\text{solv}}^{\text{np}} + \Delta G_{\text{solv}}^{\text{polar}} $ (**Eq. 1**),

where the non-polar solvation free energy $\Delta G_{\text{solv}}^{\text{np}}$ is typically estimated using the Solvent Accessible Surface Area (SASA) model, assuming proportionality to the molecule's total SASA. In contrast, the polar solvation free energy $\Delta G_{\text{solv}}^{\text{polar}}$ is commonly calculated using the Poisson–Boltzmann (PB) or Generalized Born (GB) model, which involves more complex computations.


# An example: PFOA - human serum albumin (hSA) protein

The goal of this example is to how we can use the AlphaFold 3 to predict the binding of PFOA with the hSA protein. 
To test it, I integrated all scripts (Python and Bash) together, so that we can automatically screen other potential PFAS molecules.


## Background

**Reference**
Maso, Lorenzo, et al. "Unveiling the binding mode of perfluorooctanoic acid to human serum albumin." Protein Science 30.4 (2021): 830-841. DOI: https://doi.org/10.1002/pro.4036

![Alt text](https://onlinelibrary.wiley.com/cms/asset/641b2e4e-b7a8-429b-8b78-d9238385a0ab/pro4036-fig-0001-m.jpg)

**Figure 1**. Structure of hSA in complex with PFOA and Myr. Chemical structure (top) and composite omit maps depicting the (Fo−Fc) electron density (bottom) of PFOA (a) and Myr (b) contoured at 4σ; (c) Crystal structure of hSA-PFOA-Myr complex (white) obtained using a twofold molar excess of PFOA over Myr [PDB identification code: 7AAI]; (d) Superimposition of hSA-PFOA-Myr ternary complex (white) with aligned hSA-Myr binary complex (blue white) [PDB identification code: 7AAE]. The structure of hSA is organized in homologues domains (I, II and III), subdomains (A and B), fatty acids (FA) and Sudlow's binding sites. The α-helices of hSA are represented by cylinders. Bound PFOA and Myr are shown in a ball-and-stick representation with a semi-transparent van der Waals and colored by atom type (PFOA: carbon = dark salmon, oxygen = firebrick, fluorine = palecyan; Myr: carbon = smudge green, oxygen = firebrick). The electron density PFOA and Myr is shown as grey mesh. (Note: I switched the "7AAE" with "7AAI" after checking out both structures from the PDB database.)


## Download the repository

I have created a GitHub repository, and we should have it downloaded first, which includes some required scripts. 

```bash
$ git clone https://github.com/XipingGong/pfas_docking.git
$ cd pfas_docking # go to this directory, and we will have a test later.
```


## A general script to run the gmx_MMPBSA

We will use the gmx_MMPBSA tool to calculate the binding free energy of a protein–ligand complex from a PDB structure. In our example, the PDB structure contains one protein and one ligand. To perform the gmx_MMPBSA calculations, specifically, we will apply the [single trajectory (ST) protocol](https://valdes-tresanco-ms.github.io/gmx_MMPBSA/dev/examples/Protein_ligand/ST).

It is necessary to have a look at [a ST example of gmx_MMPBSA](https://valdes-tresanco-ms.github.io/gmx_MMPBSA/dev/examples/Protein_ligand/ST). This example requires several inputs, including

**mmpbsa.in**: the input file of gmx_MMPBSA, which defines some parameters

**com.tpr**: this is tpr file from the GROMACS, which includes all possible info, including the force field, topology, etc.

**com_traj.xtc**: this is the trajectory file, which includes the coordinates of many trajectory frames. We assume that these trajectories are independently sampled and represent the major state of the protein-ligand complex.

**index.ndx**: this is the index file created from GROMACS, which includes the indices of protein and ligand "-cg 3 4" should refer to the protein and ligand in the index.ndx file.

**topol.top**: this is the topol.top file created from GROMACS, which includes all parameters, etc. 

+ **Step 1: Prepare the input file for gmx_MMPBSA**

In essential, the only input file for running a MMPBSA calculation is the 3-D PDB structrue of protein-ligand system.
Here, we use a script to directly download a native PDB file from the RCSB PDB.

```bash
# Create a folder as a working directory to run a vina job 
$ mkdir -p test/dock_dir/7AAI_8PF # create a test folder
$ cd test/dock_dir/7AAI_8PF # go to this test folder

# Obtain a native_model.pdb by using the "get_native_pdb.sh"
$ bash ../../../scripts/get_native_pdb.sh --pdbid 7AAI --ligandid 8PF # it will generate four models because of four ligands
$ cp 7AAI_8PF_1.pdb native_model.pdb # We can take the "7AAI_8PF_1.pdb" as the native structure, which is the most stable pose

# Create a folder to have native structures
$ mkdir native
$ bash ../../../scripts/get_ref_for_af3vinammpbsa.sh --input_pdb native_model.pdb --work_dir native # it will generate the native structures
# This will generate a native foler that includes all information of protein-ligand system, including the protein and ligand topology files

# Create the input files for gmx_MMPBSA
$ mkdir -p mmpbsa # create a mmpbsa folder to save the data
# Here, we take the "native_model.pdb" as an input pdb, and then we need to add hydrogens, based on the information of native structures
$ bash ../../../scripts/get_ref_for_af3vinammpbsa.sh --input_pdb native_model.pdb --work_dir mmpbsa --native_dir native # it will generate a "native_modelH.pdb" in the mmpbsa folder

```

+ **Step 2: Check out the "mmpbsa.sh" script**

```bash
# We provided a general script, "mmpbsa.sh", to run the gmx_MMPBSA.
# Please modify the INPUT section in the "scripts/mmpbsa.sh" file, like
#
# scripts_dir='/home/xg69107/program/pfas_docking/scripts' # need to change
# gmx="/home/xg69107/program/anaconda/anaconda3/envs/gmxMMPBSA/bin.AVX2_256/gmx"
# python="/home/xg69107/program/anaconda/anaconda3/bin/python"
# gmx_MMPBSA="/home/xg69107/program/anaconda/anaconda3/envs/gmxMMPBSA/bin/gmx_MMPBSA" # need to run in the gmxMMPBSA environment
#

```

+ **Step 3: Run the "mmpbsa.sh" script**

```bash

$ bash ../../../scripts/mmpbsa.sh --input_pdb mmpbsa/native_modelH.pdb --work_dir mmpbsa --native_dir native # run a MMPBSA job

```

+ **Step 4: Check out the MMPBSA energy**

```bash
# Check out three key output files
$  ls -lrt mmpbsa/*_emin*
# >> mmpbsa/native_modelH_emin.pdb # the output pdb file by a vacuum structural minimization. 
# >> mmpbsa/native_modelH_emin_MMPBSA.dat # MMPBSA data output for the mmpbsa/native_modelH_emin.pdb
# >> mmpbsa/native_modelH_emin_RMSD.dat # RMSD data output for the the mmpbsa/native_modelH_emin.pdb in terms of the input pdb file


# Check the binding free energy
$ cat mmpbsa/native_modelH_emin_MMPBSA.dat # | grep 'ΔTOTAL'
# >> -------------------------------------------------------------------------------
# >> Delta (Complex - Receptor - Ligand):
# >> Energy Component       Average     SD(Prop.)         SD   SEM(Prop.)        SEM
# >> -------------------------------------------------------------------------------
# >> ΔBOND                     0.00          0.00       0.00         0.00       0.00
# >> ΔANGLE                    0.00          0.00       0.00         0.00       0.00
# >> ΔDIHED                   -0.00          0.00       0.00         0.00       0.00
# >> ΔVDWAALS                -27.76          0.00       0.00         0.00       0.00
# >> ΔEEL                    -48.67          0.00       0.00         0.00       0.00
# >> Δ1-4 VDW                  0.00          0.00       0.00         0.00       0.00
# >> Δ1-4 EEL                  0.00          0.00       0.00         0.00       0.00
# >> ΔEGB                     59.66          0.00       0.00         0.00       0.00
# >> ΔESURF                   -5.74          0.00       0.00         0.00       0.00
# >> 
# >> ΔGGAS                   -76.44          0.00       0.00         0.00       0.00
# >> ΔGSOLV                   53.93          0.00       0.00         0.00       0.00
# >> 
# >> ΔTOTAL                  -22.51          0.00       0.00         0.00       0.00
# >> -------------------------------------------------------------------------------

# Check the RMSD values of the minimized structure
$ cat mmpbsa/native_modelH_emin_RMSD.dat 
# >> 📂 Loading reference PDB: /home/xg69107/program/pfas_docking/test/dock_dir/7AAI_8PF/native/native_modelH.pdb
# >>   - /home/xg69107/program/pfas_docking/test/dock_dir/7AAI_8PF/native/native_modelH.pdb
# >> 📂 Loading target PDB: native_modelH_emin.pdb
# >>   - /home/xg69107/program/pfas_docking/test/dock_dir/7AAI_8PF/mmpbsa/native_modelH_emin.pdb
# >> 📊 Protein Backbone RMSD (Direct): Min = 0.012 nm ; [0.01184159] nm
# >> 📊 Protein Backbone RMSD (MDTraj): Min = 0.012 nm ; [0.01174002] nm
# >> 📊 Ligand RMSD (Direct): Min = 0.019 nm ; [0.01869177] nm
# >> 📊 Ligand RMSD (MDTraj): Min = 0.018 nm ; [0.01814515] nm
# >> 📊 Protein Backbone Pocket RMSD (Direct): Min = 0.011 nm ; [0.01139026] nm
# >> 📊 Protein Backbone Pocket RMSD (MDTraj): Min = 0.011 nm ; [0.0113957] nm

```

## Analysis & Conclusion

We see the binding free energy has this relationship, and there is no contributions from bonded interactions, because we assume that the structures of protein and ligand are unchanged for the bound and unbound structures. Obviously, this ignores the entropy contributions from the structural flexibility of both protein and ligand systems, but this reduces the noise resulted from the bonded interactions.

In addition, in this gmx_MMPBSA calculation, we optimized the geometry of the input PDB structure under a vacuum calculation, in case that the input structure has a bad initial structure, which could cause an error, where the bonded energies will be very large. If you do not want to optimize, then we can modify the native/em.mdp, and set the "nsteps" to be zero. We suggest to optimize this input structure, to avoid this potential error, and also the obtained minimized structure usually has a very small RMSD value with the input PDB structure.

```text

ΔTOTAL = ΔGGAS + ΔGSOLV, 

where, ΔGGAS = ΔVDWAALS + ΔEEL, and ΔGSOLV = ΔEGB + ΔESURF.

So, the calcualted binding free energy is -22.51 kcal/mol.

Additionally, we see the initial and optimized PDB structures have a very small RMSD value.
```

# Appendix

## How to submit this "mmpbsa.sh" script at Sapelo2@GACRC?

Please also check out the documentation from here: https://wiki.gacrc.uga.edu/wiki/Running_Jobs_on_Sapelo2#Serial_(single-processor)_Job